# Фильтр Блума со счётом

## 1. Реализация Фильтра Блума

In [1]:
import hashlib
import math

class CountingBloomFilter:
    def __init__(self, size, num_hashes):
        self.size = size
        self.num_hashes = num_hashes
        self.buckets = [0] * size

    def _hashes(self, item):
        return [int(hashlib.md5(item.encode()).hexdigest(), 16) % self.size for _ in range(self.num_hashes)]

    def add(self, item):
        for hash_value in self._hashes(item):
            self.buckets[hash_value] += 1

    def remove(self, item):
        for hash_value in self._hashes(item):
            if self.buckets[hash_value] > 0:
                self.buckets[hash_value] -= 1

    def contains(self, item):
        return all(self.buckets[hash_value] > 0 for hash_value in self._hashes(item))

## 2. Определение ложноположительных результатов

In [2]:
def test_false_positive_rate(bloom_filter, num_tests, num_elements):
    false_positives = 0
    for i in range(num_tests):
        test_item = f"test_{i}"
        if not bloom_filter.contains(test_item):
            false_positives += 1
    return false_positives / num_tests

# Пример использования
bloom_filter = CountingBloomFilter(size=1000, num_hashes=5)
for i in range(100):
    bloom_filter.add(f"item_{i}")

false_positive_rate = test_false_positive_rate(bloom_filter, 1000, 100)
print(f"False positive rate: {false_positive_rate:.2%}")

False positive rate: 90.80%


## 3. Оценка зависимости реализации от гиперпараметров

In [12]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

sizes = [100, 200, 500, 1000]
num_hashes = [1, 2, 3, 4, 5]
results = []

for size in sizes:
    for num_hash in num_hashes:
        bloom_filter = CountingBloomFilter(size=size, num_hashes=num_hash)
        for i in range(100):
            bloom_filter.add(f"item_{i}")
        false_positive_rate = test_false_positive_rate(bloom_filter, 1000, 100)
        results.append((size, num_hash, false_positive_rate))

results = np.array([results])
results

array([[[1.00e+02, 1.00e+00, 3.76e-01],
        [1.00e+02, 2.00e+00, 3.76e-01],
        [1.00e+02, 3.00e+00, 3.76e-01],
        [1.00e+02, 4.00e+00, 3.76e-01],
        [1.00e+02, 5.00e+00, 3.76e-01],
        [2.00e+02, 1.00e+00, 6.08e-01],
        [2.00e+02, 2.00e+00, 6.08e-01],
        [2.00e+02, 3.00e+00, 6.08e-01],
        [2.00e+02, 4.00e+00, 6.08e-01],
        [2.00e+02, 5.00e+00, 6.08e-01],
        [5.00e+02, 1.00e+00, 8.11e-01],
        [5.00e+02, 2.00e+00, 8.11e-01],
        [5.00e+02, 3.00e+00, 8.11e-01],
        [5.00e+02, 4.00e+00, 8.11e-01],
        [5.00e+02, 5.00e+00, 8.11e-01],
        [1.00e+03, 1.00e+00, 9.08e-01],
        [1.00e+03, 2.00e+00, 9.08e-01],
        [1.00e+03, 3.00e+00, 9.08e-01],
        [1.00e+03, 4.00e+00, 9.08e-01],
        [1.00e+03, 5.00e+00, 9.08e-01]]])